### Clean data file into fit() ready spark dataframe

In [1]:
import pandas as pd
import numpy as np

In [4]:
from src_2.submit import compute_score

In [3]:
# import pyspark
# from pyspark.sql import SparkSession
# from pyspark.ml.recommendation import ALS

# # Setup a SparkSession
# spark = SparkSession.builder.getOrCreate()

In [12]:
pd_requests_df = pd.read_csv("data/requests.csv")
pd_training_df = pd.read_csv("data/training.csv").drop(['timestamp'], axis=1)
print pd_requests_df.head()
print pd_training_df.head()

   user  movie
0  4958   1924
1  4958   3264
2  4958   2634
3  4958   1407
4  4958   2399
   user  movie  rating
0  6040    858       4
1  6040    593       5
2  6040   2384       4
3  6040   1961       4
4  6040   2019       5


In [5]:
repos = pd.read_csv('data/active_projects.csv', sep='\t')
repos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228180 entries, 0 to 228179
Data columns (total 7 columns):
id             228180 non-null int64
url            228180 non-null object
owner_id       228180 non-null int64
name           228174 non-null object
language       228180 non-null object
created_at     228180 non-null object
forked_from    228180 non-null object
dtypes: int64(2), object(5)
memory usage: 12.2+ MB


In [6]:
users = pd.read_csv('data/active_users.csv', sep='\t')
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159321 entries, 0 to 159320
Data columns (total 9 columns):
id              159321 non-null int64
login           159321 non-null object
company         159289 non-null object
created_at      159321 non-null object
type            159321 non-null object
country_code    159321 non-null object
state           159321 non-null object
city            159321 non-null object
location        159319 non-null object
dtypes: int64(1), object(8)
memory usage: 10.9+ MB


In [7]:
starred = pd.read_csv('data/watchers.csv')
starred.info()

KeyboardInterrupt: 

In [26]:
spark_request_df = spark.createDataFrame(pd_requests_df)
spark_training_df = spark.createDataFrame(pd_training_df)
#spark_request_df.show()

+----+-----+
|user|movie|
+----+-----+
|4958| 1924|
|4958| 3264|
|4958| 2634|
|4958| 1407|
|4958| 2399|
|4958| 3489|
|4958| 2043|
|4958| 2453|
|5312| 3267|
|5948| 3098|
|5948| 1180|
|3158| 2648|
| 403| 1036|
|3693|  468|
|5950| 1262|
|5950| 3555|
|5950| 3793|
|5950| 3578|
|5950| 3948|
|5950| 3893|
+----+-----+
only showing top 20 rows



### Try out Surprise

# Nope

In [15]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise import evaluate, print_perf

In [16]:
pd_requests_df = pd.read_csv("data/requests.csv")
pd_training_df = pd.read_csv("data/training.csv").drop(['timestamp'], axis=1)

In [18]:
pd_training_df.head()

,user,movie,rating
0,6040,858,4
1,6040,593,5
2,6040,2384,4
3,6040,1961,4
4,6040,2019,5


In [20]:
train, test = spark.createDataFrame(pd_training_df).randomSplit([0.8, 0.2], seed=5882300)

In [22]:
train = train.toPandas()
test = test.toPandas()

In [23]:
!head 'data/training.csv'

user,movie,rating,timestamp
6040,858,4,956703932
6040,593,5,956703954
6040,2384,4,956703954
6040,1961,4,956703977
6040,2019,5,956703977
6040,1419,3,956704056
6040,573,4,956704056
6040,3111,5,956704056
6040,213,5,956704056


## ALS

In [32]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

from src.evaluate import *

# Setup a SparkSession
spark = SparkSession.builder.getOrCreate()

# # Convert a Pandas DF to a Spark DF
# spark_df = spark.createDataFrame(pandas_df) 

# # Convert a Spark DF to a Pandas DF
# pandas_df = spark_df.toPandas()

In [27]:
spark_request_df = spark.createDataFrame(pd_requests_df)
spark_training_df = spark.createDataFrame(pd_training_df)

In [28]:
spark_training_df.show()

+----+-----+------+
|user|movie|rating|
+----+-----+------+
|6040|  858|     4|
|6040|  593|     5|
|6040| 2384|     4|
|6040| 1961|     4|
|6040| 2019|     5|
|6040| 1419|     3|
|6040|  573|     4|
|6040| 3111|     5|
|6040|  213|     5|
|6040| 3505|     4|
|6040| 1734|     2|
|6040|  912|     5|
|6040|  919|     5|
|6040| 2503|     5|
|6040|  527|     5|
|6040|  318|     4|
|6040| 1252|     5|
|6040|  649|     5|
|6040| 3289|     5|
|6040|  759|     5|
+----+-----+------+
only showing top 20 rows



In [38]:
als_model = ALS(
    itemCol='movie',
    userCol='user',
    ratingCol='rating',
    nonnegative=True,    
    regParam=0.1,
    rank=10
    ) 

In [29]:
train, test = spark_training_df.randomSplit([0.8, 0.2], seed=427471138)
recommender = als_model.fit(train)
predictions = recommender.transform(test)

In [39]:
predictions_pandas = predictions.toPandas()

In [36]:
evaluate_recommender(als_model, train, test)

AttributeError: 'DataFrame' object has no attribute 'actualrating'

In [87]:
ALS_predictions = pd.read_csv('submissions/ALS.csv')
brians_mean_movie_rating = pd.read_csv('submissions/brian_avg_movie_rating.csv')

In [88]:
ALS_predictions.head()

,user,movie,prediction
0,53,148,NaN
1,4169,148,3.034759
2,5333,148,2.316155
3,4387,148,2.017670
4,840,148,2.425453


In [89]:
brians_mean_movie_rating.head()

,user,movie,rating
0,4958,1924,2.645714
1,4958,3264,3.110448
2,4958,2634,3.161290
3,4958,1407,3.637457
4,4958,2399,2.717391


In [96]:
ALS_predictions.describe()

,user,movie,prediction
count,200209.000000,200209.000000,104439.000000
mean,1511.751225,1930.586682,3.363050
std,1582.930564,1129.670350,0.664480
min,1.000000,1.000000,0.475375
25%,331.000000,1046.000000,2.950058
50%,752.000000,1946.000000,3.423546
75%,2131.000000,2890.000000,3.834524
max,6040.000000,3952.000000,5.428523


In [65]:
brians_mean_movie_rating.describe()

,user,movie,rating
count,200209.000000,200209.000000,200082.000000
mean,1511.751225,1930.586682,3.563625
std,1582.930564,1129.670350,0.550986
min,1.000000,1.000000,1.000000
25%,331.000000,1046.000000,3.241379
50%,752.000000,1946.000000,3.662857
75%,2131.000000,2890.000000,3.962349
max,6040.000000,3952.000000,5.000000


In [64]:
print "ALS Model Predicitons"
print ALS_predictions.info()
print "\n    Missing Predictions just running through ALS: {}%\n".format((104439./200209)*100)
print "Simple Predict the Mean Movie Rating"
print brians_mean_movie_rating.info()

ALS Model Predicitons
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200209 entries, 0 to 200208
Data columns (total 3 columns):
user          200209 non-null int64
movie         200209 non-null int64
prediction    104439 non-null float64
dtypes: float64(1), int64(2)
memory usage: 4.6 MB
None

    Missing Predictions just running through ALS: 52.164987588%

Simple Predict the Mean Movie Rating
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200209 entries, 0 to 200208
Data columns (total 3 columns):
user      200209 non-null int64
movie     200209 non-null int64
rating    200082 non-null float64
dtypes: float64(1), int64(2)
memory usage: 4.6 MB
None


In [68]:
ALS_join = pd.merge(ALS_predictions, brians_mean_movie_rating, how='left', on=["user","movie"])

In [69]:
ALS_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200209 entries, 0 to 200208
Data columns (total 4 columns):
user          200209 non-null int64
movie         200209 non-null int64
prediction    104439 non-null float64
rating        200082 non-null float64
dtypes: float64(2), int64(2)
memory usage: 7.6 MB


In [75]:
ALS_join['prediction'].fillna(value=ALS_join.rating, inplace=True)

In [76]:
ALS_join.head(10)

,user,movie,prediction,rating
0,53,148,2.700000,2.700000
1,4169,148,3.034759,2.700000
2,5333,148,2.316155,2.700000
3,4387,148,2.017670,2.700000
4,840,148,2.425453,2.700000
5,216,148,2.700000,2.700000
6,482,148,2.700000,2.700000
7,752,148,2.999931,2.700000
8,424,148,2.700000,2.700000
9,970,463,2.748142,2.821429


In [77]:
ALS_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200209 entries, 0 to 200208
Data columns (total 4 columns):
user          200209 non-null int64
movie         200209 non-null int64
prediction    200092 non-null float64
rating        200082 non-null float64
dtypes: float64(2), int64(2)
memory usage: 7.6 MB


In [ ]:
ALS_join = ALS_join.drop('rating', axis=1)

In [81]:
ALS_join = ALS_join.rename(columns={'prediction':'rating'})
ALS_join.head()

,user,movie,rating
0,53,148,2.700000
1,4169,148,3.034759
2,5333,148,2.316155
3,4387,148,2.017670
4,840,148,2.425453


In [84]:
ALS_join.to_csv('submissions/ALS_impute_Movie_Means1.csv', index=False)


In [86]:
!head 'submissions/ALS_impute_Movie_Means.csv'

user,movie,rating
53,148,2.7
4169,148,3.03475856781
5333,148,2.31615543365
4387,148,2.01766991615
840,148,2.42545342445
216,148,2.7
482,148,2.7
752,148,2.99993133545
424,148,2.7


In [90]:
brians_4 = pd.read_csv('submissions/brians_meanmovierating_userbias.csv')

In [92]:
brians_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200209 entries, 0 to 200208
Data columns (total 3 columns):
user      200209 non-null int64
movie     200209 non-null int64
rating    200209 non-null float64
dtypes: float64(1), int64(2)
memory usage: 4.6 MB


In [94]:
new_predictions = pd.read_csv('submissions/ALS_with_imputing435.csv')
new_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200209 entries, 0 to 200208
Data columns (total 3 columns):
user      200209 non-null int64
movie     200209 non-null int64
rating    200209 non-null float64
dtypes: float64(1), int64(2)
memory usage: 4.6 MB


In [95]:
new_predictions.describe()

,user,movie,rating
count,200209.000000,200209.000000,200209.000000
mean,1511.751225,1930.586682,3.472030
std,1582.930564,1129.670350,0.622423
min,1.000000,1.000000,0.475375
25%,331.000000,1046.000000,3.103139
50%,752.000000,1946.000000,3.555398
75%,2131.000000,2890.000000,3.920826
max,6040.000000,3952.000000,5.428523


## Model Performances

#### Predict every user rates the movie a 5
Rating of 3.54594448801

#### Predict the rating for every user using the mean movie rating for each movie
Rating of 4.27909532094

#### Predict the rating using ALS and imputing the missing values using the mean movie rating and more

Rating of 4.32845

In [97]:
4.32845 - 4.27909532094

0.04935467906000035

## Export Fork relationships to CSV

In [2]:
fork_relationships = pd.read_pickle('/Users/mac/Documents/DSI/github-collaborator/data/pickled/active_forks.pkl')
fork_relationships.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133747 entries, 3 to 228179
Data columns (total 4 columns):
user_id       133747 non-null int64
repo_id       133747 non-null int64
created_at    133747 non-null datetime64[ns]
forked        133747 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 5.1 MB
